# 📚 Fine‑Tuning a DialoGPT Chatbot (Template)
Esta _notebook_ guia você passo a passo para **criar um chatbot em domínio específico** usando *transfer learning*.

> **Fluxo geral**  
1. Instalar dependências  
2. Preparar/Carregar dataset de diálogos  
3. Tokenizar & formatar  
4. Fine‑tuning (Trainer)  
5. Avaliação & Inferência  
6. (Opcional) LoRA fine‑tuning para GPUs pequenas  
7. (Opcional) Gradio deploy

_Plug‑and‑play_: basta trocar a célula **`DATASET`** pelo seu conjunto de perguntas e respostas (culinária ou outro).

In [31]:
# !pip -q install transformers datasets accelerate bitsandbytes peft gradio --upgrade

In [32]:
import torch, platform, os, subprocess, sys, json, time, random, math
print('Python', sys.version)
print('Torch', torch.__version__)
print('GPU disponível?', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

Python 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
Torch 2.6.0+cpu
GPU disponível? False


In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline
from datasets import load_dataset
from pathlib import Path
MODEL_NAME = 'microsoft/DialoGPT-small'  # troque para medium se tiver VRAM
TOK_EOS = 50256

## 📝 1. Dataset
Crie um arquivo **`dialogs.jsonl`** onde cada linha contenha um diálogo completo já concatenado com __tokens de fim__ (ex.: ``).
```jsonl
{"text": "Pergunta? Resposta.  "}
```
Abaixo há um _mock_ para testes:

In [34]:
"""# ⚠️ SUBSTITUA por seu próprio conjunto!
mock_dialogs = [
    ["Qual a temperatura para assar lasanha?", "Em média 200 °C por 25 a 30 min."] ,
    ["Como evitar que o arroz queime?", "Use fogo baixo e panela com fundo grosso; não esqueça a proporção água/arroz."]
]
import json, textwrap, random
with open('dialogs.jsonl', 'w', encoding='utf-8') as f:
    for q, a in mock_dialogs:
        f.write(json.dumps({'text': f"{q} {a} "}) + '\n')
print('Exemplo salvo:')
!head -n 2 dialogs.jsonl"""

'# ⚠️ SUBSTITUA por seu próprio conjunto!\nmock_dialogs = [\n    ["Qual a temperatura para assar lasanha?", "Em média 200\u202f°C por 25 a 30\u202fmin."] ,\n    ["Como evitar que o arroz queime?", "Use fogo baixo e panela com fundo grosso; não esqueça a proporção água/arroz."]\n]\nimport json, textwrap, random\nwith open(\'dialogs.jsonl\', \'w\', encoding=\'utf-8\') as f:\n    for q, a in mock_dialogs:\n        f.write(json.dumps({\'text\': f"{q} {a} "}) + \'\n\')\nprint(\'Exemplo salvo:\')\n!head -n 2 dialogs.jsonl'

In [35]:
# ▶️ Tokenização
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
def tokenize(example):
    ids = tok(example['text']).input_ids
    example['input_ids'] = ids
    example['labels']    = ids.copy()
    return example

ds = load_dataset('json', data_files='culinaria_dialogs.jsonl')['train']
tok_ds = ds.map(tokenize, remove_columns=['text'])
tok_ds.set_format(type='torch', columns=['input_ids', 'labels'])
print('Samples:', len(tok_ds))

Samples: 80


In [36]:
# ⚙️ Hiperparâmetros
args = TrainingArguments(
    output_dir='chatbot_ckpt',
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    save_strategy='epoch',
)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

trainer = Trainer(model=model, args=args, train_dataset=tok_ds)
trainer.train()
trainer.save_model('chatbot_final')
tok.save_pretrained('chatbot_final')

RuntimeError: stack expects each tensor to be equal size, but got [56] at entry 0 and [50] at entry 1

## 🤖 2. Inferência interativa

In [37]:
pipe = pipeline('text-generation', model='chatbot_final', tokenizer='chatbot_final',
                       max_new_tokens=64, pad_token_id=tok.eos_token_id)

history = []
while True:
    user = input('Você: ')
    if user.lower() in {'sair','exit','quit'}:
        break
    history.append(user)
    prompt = ' '.join(history) + ' '
    resp = pipe(prompt, do_sample=True, temperature=0.8, top_p=0.9)[0]['generated_text']
    bot_reply = resp.split(prompt)[-1].strip()
    print('ChefBot:', bot_reply)
    history.append(bot_reply)

OSError: chatbot_final is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

## 📊 3. Avaliação rápida (BLEU simples em minitest)

In [38]:
from torchmetrics.text.bleu import BLEUScore
bleu = BLEUScore()
# Exemplo tosco; substitua por conjunto válido de validação
references = [tok.decode(sample['labels']) for sample in tok_ds]
preds = []
for ref in references:
    out = pipe(ref, max_new_tokens=32, do_sample=False)[0]['generated_text']
    preds.append(out)
score = bleu(preds, references)
print('BLEU ≈', score.item())

ModuleNotFoundError: No module named 'torchmetrics'

## 🔧 4. (Opcional) Fine‑tuning com LoRA
Para VRAM limitada, use `peft` + `bitsandbytes`. Exemplo:

In [39]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, load_in_4bit=True)
base_model = prepare_model_for_kbit_training(base_model)
config = LoraConfig(r=8, lora_alpha=32, target_modules=['c_attn'], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
lora_model = get_peft_model(base_model, config)
print('Parâmetros treináveis:', sum(p.numel() for p in lora_model.parameters() if p.requires_grad))

ModuleNotFoundError: No module named 'peft'

## 🌐 5. Deploy rápido com Gradio

In [40]:
import gradio as gr

chatbot_pipe = pipeline('text-generation', model='chatbot_final', tokenizer='chatbot_final',
                        max_new_tokens=64, pad_token_id=tok.eos_token_id)

def chat_fn(message, history):
    if history is None:
        history = []
    history.append(message)
    prompt = ' '.join(history) + ' '
    reply = chatbot_pipe(prompt, do_sample=True, temperature=0.7, top_p=0.9)[0]['generated_text']
    bot_reply = reply.split(prompt)[-1].strip()
    history.append(bot_reply)
    return history, history

gr.ChatInterface(chat_fn, title='ChefBot Demo').launch(debug=False, share=False)

ModuleNotFoundError: No module named 'gradio'